# Pytorch로 딥러닝 제대로 배우기-중급
## Part6-4: CNN 실습 (2)
### 목차
1. 데이터 호출(Cifar-10)
2. 학습/평가함수 정의
3. CNN 응용
4. Sequential vs. layer by layer

### (1) 데이터 호출 (Cifar-10)

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose

In [2]:
## Transform 지정 - ** 초급강좌에서 설명 **
transform = Compose(
    [ToTensor(),
     Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Batch size와 학습의 관계? 
batch_size = 64

trainset = datasets.CIFAR10(root='./dataset', train=True,
                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./dataset', train=False,
                           download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### (2) 학습/평가 함수 정의

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [4]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### (3) CNN 응용

In [5]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1_3x3 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3)
    self.conv1_5x5 = nn.Conv2d(3, 6, 5, padding=1) 
    self.conv1_7x7 = nn.Conv2d(3, 6, 7, padding=2)  
    self.pool = nn.MaxPool2d(2, 2)

    self.conv2 = nn.Conv2d(18, 18, 1)
    
    # FC layers
    self.fc1 = nn.Linear(18*7*7, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    x_3x3 = self.pool(F.relu(self.conv1_3x3(x)))
    x_5x5 = self.pool(F.relu(self.conv1_5x5(x)))
    x_7x7 = self.pool(F.relu(self.conv1_7x7(x)))
    x = torch.cat((x_3x3, x_5x5, x_7x7), dim=1)

    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    logits = self.fc3(x)
    return logits


model = Net().to(device)
print(model)

Net(
  (conv1_3x3): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv1_5x5): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv1_7x7): Conv2d(3, 6, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(18, 18, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=882, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### Weights 살펴보기

In [6]:
for k, v in model.state_dict().items():
  print(k)
  print(v.size())

conv1_3x3.weight
torch.Size([6, 3, 3, 3])
conv1_3x3.bias
torch.Size([6])
conv1_5x5.weight
torch.Size([6, 3, 5, 5])
conv1_5x5.bias
torch.Size([6])
conv1_7x7.weight
torch.Size([6, 3, 7, 7])
conv1_7x7.bias
torch.Size([6])
conv2.weight
torch.Size([18, 18, 1, 1])
conv2.bias
torch.Size([18])
fc1.weight
torch.Size([120, 882])
fc1.bias
torch.Size([120])
fc2.weight
torch.Size([84, 120])
fc2.bias
torch.Size([84])
fc3.weight
torch.Size([10, 84])
fc3.bias
torch.Size([10])


### (4) 학습

#### Adam

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304695  [    0/50000]
loss: 1.703658  [ 6400/50000]
loss: 1.823117  [12800/50000]
loss: 1.499407  [19200/50000]
loss: 1.571092  [25600/50000]
loss: 1.249847  [32000/50000]
loss: 1.188116  [38400/50000]
loss: 1.388546  [44800/50000]
Test Error: 
 Accuracy: 49.1%, Avg loss: 1.408463 

Epoch 2
-------------------------------
loss: 1.686388  [    0/50000]
loss: 1.142175  [ 6400/50000]
loss: 1.424866  [12800/50000]
loss: 1.276735  [19200/50000]
loss: 1.251418  [25600/50000]
loss: 1.374383  [32000/50000]
loss: 1.246162  [38400/50000]
loss: 1.360692  [44800/50000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 1.205323 

Epoch 3
-------------------------------
loss: 1.306747  [    0/50000]
loss: 1.167632  [ 6400/50000]
loss: 0.949511  [12800/50000]
loss: 1.321633  [19200/50000]
loss: 0.940683  [25600/50000]
loss: 1.300548  [32000/50000]
loss: 1.378575  [38400/50000]
loss: 0.937053  [44800/50000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.133009 

Epoc

### 4. Sequential vs. Layer by Layer

#### Sequential blocks

In [9]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.block1 = nn.Sequential(
        nn.Conv2d(3, 6, 3),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(6, 12, 3),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.block2 = nn.Sequential(
        nn.Conv2d(6, 12, 3),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )

    # FC
    self.fc1 = nn.Linear(12 * 6 * 6, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self, x):
    x = self.block1(x)
    x = self.block2(x)
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = Net().to(device)
print(model)

Net(
  (block1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=432, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [10]:
for k, v in model.state_dict().items():
  print(k)
  print(v.size())

block1.0.weight
torch.Size([6, 3, 3, 3])
block1.0.bias
torch.Size([6])
block1.3.weight
torch.Size([12, 6, 3, 3])
block1.3.bias
torch.Size([12])
block2.0.weight
torch.Size([12, 6, 3, 3])
block2.0.bias
torch.Size([12])
fc1.weight
torch.Size([120, 432])
fc1.bias
torch.Size([120])
fc2.weight
torch.Size([84, 120])
fc2.bias
torch.Size([84])
fc3.weight
torch.Size([10, 84])
fc3.bias
torch.Size([10])


#### Layer by Layer

In [11]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 3)
    self.conv2 = nn.Conv2d(6, 12, 3)
    self.pool = nn.MaxPool2d(2, 2)
    
    # FC
    self.fc1 = nn.Linear(12 * 6 * 6, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=432, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [12]:
for k, v in model.state_dict().items():
  print(k)
  print(v.size())

conv1.weight
torch.Size([6, 3, 3, 3])
conv1.bias
torch.Size([6])
conv2.weight
torch.Size([12, 6, 3, 3])
conv2.bias
torch.Size([12])
fc1.weight
torch.Size([120, 432])
fc1.bias
torch.Size([120])
fc2.weight
torch.Size([84, 120])
fc2.bias
torch.Size([84])
fc3.weight
torch.Size([10, 84])
fc3.bias
torch.Size([10])
